# SQLite Python: Inserting Data
Reference: https://www.sqlitetutorial.net/sqlite-python/

In [4]:
import pandas as pd
import sqlite3
import sqlalchemy
from IPython.display import display


### Create a new SQLite database
If there is the menu_data.db file, then this function will connect to the database.
If the menu_data.db file doesn't exist, then this function will create a new database file.


In [66]:
def create_sqlite_database(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(db_file, "connected/created successfully")
        print("SQLite version:", sqlite3.version)
    except sqlite3.Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_sqlite_database(r"menu_data.db")

menu_data.db connected/created successfully
SQLite version: 2.6.0


### Use the create_connection function, the create_table function, and the main function
### to create tables, define attribute domains, and define constrains
Note that, tables are defined and created, don't have to run this chunk of code

In [67]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)

### Create the main function to create tables
def main():
    sql_create_restaurant_table = """ CREATE TABLE IF NOT EXISTS restaurant (
                                        r_id integer PRIMARY KEY,
                                        r_name text NOT NULL
                                    ); """

    sql_create_food_category_table = """CREATE TABLE IF NOT EXISTS food_category (
                                    cat_id integer PRIMARY KEY,
                                    cat_name text NOT NULL
                                );"""

    sql_create_item_info_table = """CREATE TABLE IF NOT EXISTS item_info (
                                    item_id integer NOT NULL,
                                    year integer NOT NULL,
                                    item_name text NOT NULL,
                                    item_description text,
                                    PRIMARY KEY (item_id, year)
                                );"""

    sql_create_nutrition_facts_table = """CREATE TABLE IF NOT EXISTS nutrition_facts (
                                    item_id integer NOT NULL,
                                    year integer NOT NULL,
                                    r_id integer,
                                    cat_id integer,
                                    calories integer,
                                    total_fat integer,
                                    saturated_fat integer,
                                    trans_fat integer,
                                    cholesterol integer,
                                    sodium integer,
                                    potassium integer,
                                    carbs integer,
                                    protein integer,
                                    sugar integer,
                                    dietary_Fiber integer,
                                    PRIMARY KEY (item_id, year),
                                    FOREIGN KEY (item_id) REFERENCES item_info (item_id),
                                    FOREIGN KEY (year) REFERENCES item_info (year),
                                    FOREIGN KEY (r_id) REFERENCES restaurant (r_id),
                                    FOREIGN KEY (cat_id) REFERENCES food_category (cat_id)
                                );"""

    sql_create_combo_facts_table = """CREATE TABLE IF NOT EXISTS combo_facts (
                                    combo_id integer NOT NULL,
                                    item_id integer NOT NULL,
                                    year integer NOT NULL,
                                    cat_id integer,
                                    builds text,
                                    PRIMARY KEY (combo_id, item_id, year, cat_id),
                                    FOREIGN KEY (item_id) REFERENCES item_info (item_id),
                                    FOREIGN KEY (year) REFERENCES item_info (year),
                                    FOREIGN KEY (cat_id) REFERENCES food_category (cat_id)
                                );"""

    # create a database connection
    conn = create_connection(r"menu_data.db")

    # create tables
    if conn is not None:
        # create tables
        try:
            create_table(conn, sql_create_restaurant_table)
            create_table(conn, sql_create_food_category_table)
            create_table(conn, sql_create_item_info_table)
            create_table(conn, sql_create_nutrition_facts_table)
            create_table(conn, sql_create_combo_facts_table)
        except:
            print("Error! cannot create the database connection.")
        finally:
            if conn:
                conn.close()

if __name__ == '__main__':
    main()

### Testing: check tables

In [68]:
conn = None
sql = """SELECT name FROM sqlite_master WHERE type='table';"""
sql2 = """SELECT sql
        FROM sqlite_master
        WHERE name = 'nutrition_facts'"""
sql3 = """DROP TABLE combo_facts"""
try:
    conn = sqlite3.connect(r"menu_data.db")
    if conn is not None:
        cursor = conn.cursor()
        cursor.execute(sql)
        print(cursor.fetchall())
        cursor.close()
        conn.close()
except sqlite3.Error as e:
    print(e)

[('restaurant',), ('food_category',), ('item_info',), ('nutrition_facts',), ('combo_facts',)]


# Exporting pandas DataFrames into SQLite with SQLAlchemy, Appending DataFrame to DB Tables
Reference: https://www.fullstackpython.com/blog/export-pandas-dataframes-sqlite-sqlalchemy.html

### Funtions used to append pandas dataframe into the target table

In [69]:
def append_table(db_file, tbl_name, pd_df):
    """
    Use SQLAlchemy to append pandas dataframe to the target table in the SQLite db
    :param db_file: SQLite db file name
    :param tbl_name: SQLite db table name, target table
    :param pd_df: Pandas dataframe name, the exporting df
    :return:
    """
    db_path = 'sqlite:///' + db_file
    engine = sqlalchemy.create_engine(db_path, echo=True)
    sqlite_connection = engine.connect()
    sqlite_table = tbl_name
    pd_df.to_sql(sqlite_table, sqlite_connection, if_exists='append', index = False)
    sqlite_connection.close()

def load_data(year, file_name, target_table):
    """
    calls append_table() to load data
    :param year: the year of the dataset
    :param file_name: file name
    :param target_table:
    :return:
    """
    path = 'cleaned_tables_by_year/' + year + '/' + file_name
    dat = pd.read_csv(path)
    append_table(db_file="menu_data.db", tbl_name=target_table, pd_df=dat)

## Do not run the load data code again, would duplicate the data

In [70]:
#load_data(year='2018', file_name='food_category.csv', target_table='food_category')
#load_data(year='2018', file_name='restaurant.csv', target_table='restaurant')
#load_data(year='2018', file_name='item_info.csv', target_table='item_info')
#load_data(year='2018', file_name='nutrition_facts.csv', target_table='nutrition_facts')
#load_data(year='2018', file_name='combo_facts.csv', target_table='combo_facts')

In [71]:
#load_data(year='2017', file_name='item_info.csv', target_table='item_info')
#load_data(year='2017', file_name='nutrition_facts.csv', target_table='nutrition_facts')
#load_data(year='2017', file_name='combo_facts.csv', target_table='combo_facts')

In [72]:
#load_data(year='2016', file_name='item_info.csv', target_table='item_info')
#load_data(year='2016', file_name='nutrition_facts.csv', target_table='nutrition_facts')
#load_data(year='2016', file_name='combo_facts.csv', target_table='combo_facts')

In [73]:
#load_data(year='2015', file_name='item_info.csv', target_table='item_info')
#load_data(year='2015', file_name='nutrition_facts.csv', target_table='nutrition_facts')
# Failed to load 2015 combo_facts data

In [74]:
#load_data(year='2014', file_name='item_info.csv', target_table='item_info')
#load_data(year='2014', file_name='nutrition_facts.csv', target_table='nutrition_facts')
# Failed to load 2014 combo_facts data

In [75]:
#load_data(year='2013', file_name='item_info.csv', target_table='item_info')
#load_data(year='2013', file_name='nutrition_facts.csv', target_table='nutrition_facts')
# Failed to load 2013 combo_facts data

In [76]:
#load_data(year='2012', file_name='item_info.csv', target_table='item_info')
#load_data(year='2012', file_name='nutrition_facts.csv', target_table='nutrition_facts')
#load_data(year='2012', file_name='combo_facts.csv', target_table='combo_facts')

In [77]:
#load_data(year='2010', file_name='item_info.csv', target_table='item_info')
#load_data(year='2010', file_name='nutrition_facts.csv', target_table='nutrition_facts')
# Failed to load 2010 combo_facts data

In [78]:
#load_data(year='2008', file_name='item_info.csv', target_table='item_info')
#load_data(year='2008', file_name='nutrition_facts.csv', target_table='nutrition_facts')
#load_data(year='2008', file_name='combo_facts.csv', target_table='combo_facts')

#### Note:
Failed to load combo_facts data for year 2015, 2014, 2013, and 2010.
Which is ok, we only need the latest menu data (2018) if we choose to analyst the combo data.

## The query_as_df function is used to display the SQL query result table

In [3]:
def query_as_df(db_file, sql):
    """
    Display SQLite db query results as a pandas df
    :param db_file: SQLite db file name
    :param sql: the sql query in this form '''query'''
    :return:
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        if conn is not None:
            df = pd.read_sql(sql, con = conn)
            conn.close()
    except sqlite3.Error as e:
        print(e)
    display(df)

### Testing database connection and the target table with imported data

In [80]:
sql = '''
SELECT * FROM food_category;
'''
query_as_df("menu_data.db", sql)

,cat_id,cat_name
0,0,Appetizers & Sides
1,1,Baked Goods
2,2,Beverages
3,3,Burgers
4,4,Desserts
5,5,Entrees
6,6,Fried Potatoes
7,7,Pizza
8,8,Salads
9,9,Sandwiches


In [81]:
sql = '''
SELECT * FROM restaurant;
'''
query_as_df("menu_data.db", sql)

,r_id,r_name
0,0,7 Eleven
1,1,Applebee's
2,2,Arby's
3,3,Auntie Anne's
4,4,BJ's Restaurant & Brewhouse
...,...,...
91,91,Whataburger
92,92,White Castle
93,93,Wingstop
94,94,Yard House


In [82]:
sql = '''
SELECT * FROM item_info;
'''
query_as_df("menu_data.db", sql)


,item_id,year,item_name,item_description
0,100054,2018,Burrito Bowl,"Burrito Bowl, Burritos & More, Vegetarian Eats..."
1,100057,2018,Quesadilla Kids Meal,"Quesadilla Kids Meal w/ Quesadilla, Applesauce..."
2,100060,2018,Burrito,"Burrito, Burritos & More, Vegetarian Eats; Cho..."
3,100073,2018,"Flour Tortilla, 12.5 in","Flour Tortilla, 12.5 in, Ingredients for Entrees"
4,10012,2018,French Onion Soup,"French Onion Soup w/ Melted Cheese & Croutons,..."
...,...,...,...,...
209527,989,2008,"Breakfast Toaster w/ Sausage, Egg & Cheese","Breakfast Toaster w/ Sausage, Egg & Cheese, Br..."
209528,99121,2008,Frontega Chicken on Focaccia,"Frontega Chicken on Focaccia, Hot Panini, From..."
209529,996,2008,Breakfast Sausage Bagel,"Breakfast Sausage Bagel, Breakfast"
209530,997,2008,Breakfast Bacon Bagel,"Breakfast Bacon Bagel, Breakfast"


In [6]:
sql = '''
SELECT * FROM nutrition_facts;
'''
query_as_df("menu_data.db", sql)


,item_id,year,r_id,cat_id,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,potassium,carbs,protein,sugar,dietary_Fiber
0,100054,2018,72.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100057,2018,72.0,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100060,2018,72.0,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100073,2018,72.0,11,300.0,7.0,3.0,0.0,0.0,760.0,50.0,52.0,8.0,2.0,3.0
4,10012,2018,1.0,10,380.0,22.0,13.0,0.0,60.0,1390.0,NaN,27.0,17.0,7.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209527,989,2008,81.0,9,620.0,42.0,13.0,1.0,340.0,1380.0,NaN,40.0,20.0,6.0,2.0
209528,99121,2008,65.0,9,810.0,34.0,9.0,5.0,100.0,2140.0,NaN,79.0,45.0,6.0,5.0
209529,996,2008,83.0,9,576.0,295.0,115.0,0.0,212.0,1120.0,NaN,51.0,26.0,6.0,2.0
209530,997,2008,83.0,9,458.0,165.0,7.0,0.0,193.0,1102.0,NaN,51.0,25.0,7.0,2.0


In [7]:
sql = '''
SELECT * FROM combo_facts;
'''
query_as_df("menu_data.db", sql)


,combo_id,item_id,year,cat_id,builds
0,704214541,100054,2018,5,Main Item
1,70814571,100057,2018,9,Main Item
2,704214551,100060,2018,9,Main Item
3,2301341,10012,2018,10,Accompanying Item
4,2331351,10012,2018,10,Accompanying Item
...,...,...,...,...,...
150966,16822481,86031,2008,11,Accompanying Item
150967,1038710163,90970,2008,4,Main Item
150968,1038710164,90972,2008,4,Main Item
150969,108181102,91453,2008,5,Main Item


In [4]:
!jupyter nbconvert load_data.ipynb --to html

[NbConvertApp] Converting notebook load_data.ipynb to html
[NbConvertApp] Writing 620991 bytes to load_data.html


# From SQLite to MySQL
- Run the following comand in the terminal to import tables over to MySQL
- you might need to change the current working directory to the path of the sqlite.db
- or specify the path of the sqlite.db in -f

reference: https://pypi.org/project/sqlite3-to-mysql/

#### sqlite3mysql -f menu_data.db -t combo_facts -d menu_data -u root -p
- -f, --sqlite-file PATH
- -t, --sqlite-tables TEXT
- -d, --mysql-database TEXT
- -u, --mysql-user TEXT
- -p, --prompt-mysql-password